<a href="https://colab.research.google.com/github/yj9889/ESAA2/blob/main/220520TMDB5000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬 머신러닝 완벽 가이드
ch9. 5 pg. 579~590

# **9.5 컨텐츠 기반 필터링 실습 – TMDB 5000 Movie Dataset**

영화 데이터 정보 사이트 IMDB의 많은 영화 중 주요 5000개 영화에 대한 메타 정보를 가공

## **장르 속성을 이용한 영화 콘텐츠 기반 필터링**
  영화를 구성하는 다양한 콘텐츠(장르, 감독, 배우, 평점, 키워드, 영화 설명) 중 **장르** 속성 기반
  
  장르 칼럼 값의 유사도를 비교한 뒤 높은 평점을 가지는 영화를 추천할 것.




## **데이터 로딩 및 가공**
  4803개 레코드, 20개 피처

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies =pd.read_csv('/content/drive/MyDrive/data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


* id 
* title - 영화제목
* genres - 영화가 속한 장르
* vote_average - 평균 평점
* vote_count - 평균 평점 투표 수
* popularity - 인기
* keyword - 주요 키워드 문구
* overview - 개요 설명

In [2]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

genres, keywords: 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열로 표기되어 있는 문제.

In [3]:
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


genres, keywords의 문자열 분해해 개별 장르를 리스트 객체로 추출

  - ast 모듈의 *literal_eval()* 함수 이용 -> list[dict1, dict2] 객체로 만듦

  - *apply()*에 *literal_eval* 적용해 문자열을 객체로 반환

In [4]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [5]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...","[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp..."


genres에서 장르명만 리스트 객체로 추출

  * 'name'키에 해당하는 값을 추출하기 위해 *apply lambda* 적용 -> 여러 딕셔너리의 'name'키에 해당하는 값을 찾아 리스트 객체로 반환

In [6]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


## **장르 콘텐츠 유사도 측정**
* genres를 문자열로 변경 -> CountVectorizer로 피처벡터화한 행렬 데이터 값을 코사인 유사도로 비교
  * 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환
  * genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교하기 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체 생성
  * 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화 추천

* 리스트 객체로 구성된 genres 칼럼을 개별 요소를 공백 문자로 구분하는 문자열로 변환해 genres_listeral 칼럼에 저장
  * 리스트 객체 내의 개별 값을 연속된 문자열로 변환하기 위해 *('구분문자').join(리스트 객체)*


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


4803개의 레코드와 276개의 개별 단어 피처로 구성된 피처 벡터 행렬 생성
* *cosine_similarity()*: 기준 행과 비교 행의 코사인 유사도를 행렬 형태로 반환

### <결과>

* *genre_sim*: movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬(genre_mat) 데이터의 행별 유사도 정보 
=> movies_df의 행별 장르 유사도 값을 가짐.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


* *argsort()[:, ::-1]* : 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값 얻을 수 있음

### <결과>
* 0번 레코드의 경우 3494 > 813 > 2401 순으로 유사도가 높음.

* *genre_sim_sorted_ind*: 각 레코드의 장르 코사인 유사도가 높은 순으로 정렬된 타 레코드의 위치 인덱스 값 가짐

In [9]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


## **장르 콘텐츠 필터링을 이용한 영화 추천**

* df: 기반 데이터
* sorted_ind: 레코드별 장르 코사인 유사도 인덱스
* title_name: 고객이 선정한 추천 기준이 되는 영화 제목
* top_n: 추천할 영화 건수(기본값: 10) 

In [10]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

### <결과>

* 2731, 1847: 비슷한 영화, 추천할 만 함
* 3636, 4065, 4217: 낮은 평점 => 개선 필요

In [11]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


vote_average: outlier로 인해 왜곡된 데이터 가질 수 있음.

3519, 4247 등 vote_count가 매우 적은(유명하지 않은) 영화가 상위권에 위치

In [12]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


* **가중 평점(Weighted Rating)**
### (v/(v+m)) * R + (m/(v+m)) * c
  * v: 개별 영화에 평점을 투표한 횟수 
    * movies_df 의 vote_count
  * m: 평점을 부여하기 위한 최소 투표 횟수
    * movies_df['vote_average'].mean(), m값을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점(여기서는 60%)
  * R: 개별 영화에 대한 평균 평점 
    * vote_average
  * C: 전체 영화에 대한 평균 평점

In [13]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


weighted_vote: 가중평균으로 구한 새로운평점 정보

In [14]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) 

In [15]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',
                                                                          ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정한 뒤, weighted_vote 칼럼 값이 높은 순으로 top_n만큼 추출하는 방식으로 변경

In [16]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


장르만으로 영화가 전달하는 모든 요소를 설명할 순 없지만, 가장 나은 결과 도출.